# Serial Python

Before we get into parallel processing, let's first consider how the type of problems we'll consider are typical solved in a serial context. We'll focus on problems which always have the following form: 

In [ ]:
def do_science(x):
    """For example:
    - training a neural network (hyperparameter tuning!)
    - getting results from a database
    - scraping some websites
    - read files
    - sampling monte-carlo style
    """
    return x ** 2  # we don't really do anything ;)

results = []
input_data = range(10)
for x in input_data:
    results.append(do_science(x))
print(results)

This typical structure (or "smell" ;) ) is pretty common and most likely all of you have something similar somewhere in your code. It's an excellent opportunity for leveraging parallelism to speed things up. However, first we'll rewrite this code using the builtin `map` function, it makes the code more compact and it will be easier to make this run in parallel. `map` takes a function and an iterable and applies the function to each iterable:

In [ ]:
results = map(do_science, input_data)
print(list(results))

(watch out: `map` returns an iterator (->advanced python), you need to convert it to a list explicitly).

### Questions?

# Embarassingly-parallel Python
This type of problem is referred to as "embarrassingly parallel" problems. This indicates that they can be easily parallelized across threads or processes as they do not require interaction while running (they can also be run in serial!). For these types of problems, we can use the builtin `multiprocessing` module. It supports parallel versions of `map` which can be run either in parallel threads or parallel processes.

## Parallel threads
We first work with the `ThreadPool` available from the `multiprocessing.pool` module. We assume CPython in which the GIL prevent several threads from executing in parallel. However, for some use case, in particular those which are **I/O bound**, threading can be very useful. Consider for example obtaining data from some database: you would like to query a couple of measurements, and completing each of these queries may take some processing time on the server. Here we mimick this server-side processing time by merely sleeping.

In [ ]:
from multiprocessing.pool import ThreadPool
import time

In [ ]:
def query_database(x):
    """Query your database to retrieve awesome measurements."""
    print(f'querying data {x} start')
    time.sleep(x)  # mimicks (input-dependent) server-side processing
    y = x ** 2
    print(f'querying data {x} end')
    return y

In [ ]:
l = [1, 8, 1.5, 2]  # some dummy queries

First, we use the builtin `map` function to perform the database query for each item in l:

In [ ]:
%%time
result = list(map(query_database, l))
print(result)

Observations?
- queries processed in serial, one after the other
- total duration is the sum of the duration of each query 
- sleep (like I/O bound tasks) does not cause high CPU load

Now, we use use `ThreadPool` do perform these queries using two threads (here the `processes` argument actually refers to the number of threads):

In [ ]:
%%time
with ThreadPool(processes=2) as pool:  # context manager providing a `ThreadPool` instance
    result = pool.map(query_database, l)
print(result)

Observations?
- results are identical to serial processing of queries; good!
- total duration is reduced: work (here: waiting for results) is distributed across threads
- allocation: queries are performed in order; thread 0 works on query 0, thread 1 on query 1, thread 0 on the rest while thread 1 is busy with query 1
- caveat: printing might get messy when printing from many threads
- caveat: optimal number of threads may be difficult to determine (more threads also means for switching!)
- caveat: load is not automatically balanced (`ThreadPool` can not know how long each query takes); in our example if long query is the last, total duration increases

In [ ]:
l = [1, 1.5, 2, 8]  # some dummy queries

In [ ]:
%%time
with ThreadPool(processes=2) as pool:
    result = pool.map(query_database, l)
print(result)

### Questions?

## Exercise
A once-in-a-lifetime opportunity presents itself: you have access to a database holding secret information about ASPP. You can sent queries to it using the `query_secret_aspp_database` function, which expects a single (arbitrary) argument. First, implement a serial version, processing several (4+) queries. Then use the `ThreadPool` to parallelize your code.

In [ ]:
from secret_aspp_database import query_secret_aspp_database

In [ ]:
%%time
# TODO fill me with the serial version
queries = ['42', 3.1415, 8162371823.123, 'pelita', 'ajwdiauwdiuahwiduhawdiuhawd']
list(map(query_secret_aspp_database, queries))

In [ ]:
%%time
with ThreadPool(processes=2) as pool:
    result = pool.map(query_secret_aspp_database, queries)
result

# Thread-parallel(?) number crunching
Now let's consider a compute-intense number-crunching task, for example tuning hyperparameters our fancy neural network model to squeeze out the additional 0.0002% increase in accuracy. Here we mimick the training by merely counting down from a large number (let's avoid cognitive overhead).

In [ ]:
def train_neural_network(x):
    """Train your favourite neural network model."""
    print(f'training with {x} start')
    n = x * 2e7  # mimick compute-intense training
    while n > 0:
        n -= 1
    y = x ** 2
    print(f'training with {x} end')
    return y

In [ ]:
l = [1, 8, 1.5, 2]  # some dummy simulations

Again, first, we use the builtin `map` function to perform the number crunching serially for each item in l:

In [ ]:
%%time
result = list(map(train_neural_network, l))
print(result)

Observations:
- number crunching causes high CPU load (surprise! ;) )

Now, we use again use `ThreadPool` do parallelize this work using two threads:

In [ ]:
%%time
with ThreadPool(processes=2) as pool:
    result = pool.map(train_neural_network, l)
print(result)

Observations?
- runtime (almost) identical to serial execution: GIL prevents simultaneous number crunching. :'(

# Parallel processes
For such tasks, we use the `ProcessPool`. In contrast to the `ThreadPool` this distributes work across multiple processes running separate instances of the Python interpreter. This allows you to circumvent the limitations of the GIL and achieve truly parallel code execution. For use cases which are **compute bound**, it is an excellent, simple-to-use option. As already introduced above, these use cases may include numerical simulations, sampling methods etc. Unfortunately, using multiple processes introduces some downsides, such as some overhead (time & memory) for launching processes and increased memory consumption (e.g., duplication of data; warning: depends on implementation and use case).

In [ ]:
from multiprocessing.pool import Pool as ProcessPool

In [ ]:
%%time
with ProcessPool(processes=2) as pool:  # context manager providing a `Pool` instance
    result = pool.map(train_neural_network, l)
print(result)

Observations?
- result are identical to serial and threaded execution; good!
- runtime is reduced compare to both serial and threaded execution
- increased CPU load on multiple cores
- caveat: as before, no automatic load balancing, tasks are executed in order

### Parallel speedup
So how much faster does my code become when I'm increasing the number of processes? Here we investiate the relative speedup (T_parallel / T_serial) for an increasing number of processes. We use the same compute-bound function as before, but remove some of the annoying output and make it a bit shorter.

In [ ]:
import time

def train_neural_network(x):
    """Train your favourite neural network model."""
    n = x * 1e6  # mimick compute-intense training
    while n > 0:
        n -= 1
    y = x ** 2
    return y


l = [2] * 16  # some dummy simulations of equal duration
times = []
for n_processes in range(1, 9):
    t0 = time.time()
    with ProcessPool(processes=n_processes) as pool:
        result = pool.map(train_neural_network, l)
    times.append(time.time() - t0)

times

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

times = np.array(times)
plt.plot(range(1, 9), times[0] / times, marker='o', label='measured')
plt.plot(range(1, 9), range(1, 9), color='k', label='ideal')
plt.xlabel(r'$n$ processes')
plt.ylabel('relative speedup')
plt.legend()

Observations
- perfect speedup with 2-3 processes, good speedup until ~4-5 processes with decreasing benefits
- no (significant) benefits for more processes
- rule of thumb: benefits up to number of cores (OS also needs some compute: context switching; also hyperthreading does not seem to work well in my experience)

# Exercise
Implement your own compute-bound function. Compare it's runtime for different numbers of processes, similar as in our example above. In particular investigate the case of "fast" compute-bound functions. Discuss your results.

In [ ]:
def crunch_numbers(x):
    pass  # TODO replace me with some number crunching

l = None  # TODO replace with meaningful input to your function
times = []
for n_processes in range(1, 9):
    t0 = time.time()
    with ProcessPool(processes=n_processes) as pool:
        result = pool.map(crunch_numbers, l)
    times.append(time.time() - t0)

times

## Resources
https://www.youtube.com/watch?v=AG1soUh4-nU
